In [1]:
import pandas as pd

In [2]:
from nsepy import get_history as gh
import datetime as dt

In [3]:
"""start = dt.datetime(2013,6,1)
end = dt.datetime(2022,2,11)
stk_data = gh(symbol='TATACOFFEE',start=start,end=end)"""


"start = dt.datetime(2013,6,1)\nend = dt.datetime(2022,2,11)\nstk_data = gh(symbol='TATACOFFEE',start=start,end=end)"

In [4]:
dataset=pd.read_csv("Tatacoffee13_21.csv", parse_dates=['Date'], index_col='Date')

#Start date and end_date
from datetime import date, timedelta
import pandas as pd
start_date = pd.to_datetime("2013-01-01")
end_date = pd.to_datetime("2019-02-04") - timedelta(days=1) #Excluding last

#List of all dates
all_date = pd.date_range(start_date, end_date, freq='d')

stk_data=dataset
stk_data.index=all_date

stk_data

,Open,High,Low,Close
2013-01-01,1410.60,1427.90,1408.30,1415.10
2013-01-02,1421.00,1626.60,1416.15,1607.40
2013-01-03,1632.55,1673.90,1613.05,1626.20
2013-01-04,1627.75,1627.75,1574.60,1579.05
2013-01-05,1580.00,1639.50,1565.50,1595.65
...,...,...,...,...
2019-01-30,202.90,207.80,201.35,205.00
2019-01-31,206.00,206.85,202.05,202.95
2019-02-01,203.90,203.90,199.35,201.00
2019-02-02,200.00,222.00,196.00,218.35


In [5]:
stk_data=stk_data[["Open","High","Low","Close"]]
#stk_data.to_csv("Tatacoffee13_21.csv")

In [6]:
#column="Close"

In [7]:
from sklearn.preprocessing import MinMaxScaler
Ms = MinMaxScaler()
data1= Ms.fit_transform(stk_data)
print("Len:",data1.shape)

Len: (2225, 4)


In [8]:
data1=pd.DataFrame(data1,columns=["Open","High","Low","Close"])

In [9]:
training_size = round(len(data1 ) * 0.80)
print(training_size)
X_train=data1[:training_size]
X_test=data1[training_size:]
print("X_train length:",X_train.shape)
print("X_test length:",X_test.shape)
y_train=data1[:training_size]
y_test=data1[training_size:]
print("y_train length:",y_train.shape)
print("y_test length:",y_test.shape)

1780
X_train length: (1780, 4)
X_test length: (445, 4)
y_train length: (1780, 4)
y_test length: (445, 4)


In [10]:
import warnings
warnings.filterwarnings("ignore")

In [11]:
performance={"Model":[],"RMSE":[],"MaPe":[],"Lag":[],"Test":[]}

In [12]:
def cominbation(dataset,listt):
    print(listt)
    datasetTwo=dataset[listt]
    test_obs = 28
    train =datasetTwo[:-test_obs]
    test = datasetTwo[-test_obs:]
    from statsmodels.tsa.api import VAR
    for i in [1,2,3,4,5,6,7,8,9,10]:
        model = VAR(train)
        results = model.fit(i)
        print('Order =', i)
        print('AIC: ', results.aic)
        print('BIC: ', results.bic)
        print()
    x = model.select_order(maxlags=12)
    order=x.selected_orders["aic"]
    result = model.fit(order)
    #result.summary()
    lagged_Values = train.values[-order:]
    pred = result.forecast(y=lagged_Values,steps=28) 
    preds=pd.DataFrame(pred,columns=listt)
    preds.to_csv("varforecasted_{}.csv".format(test_obs))
    from sklearn.metrics import mean_squared_error
    rmse= round(mean_squared_error(test,pred,squared=False))
    from sklearn.metrics import mean_absolute_percentage_error
    mape=mean_absolute_percentage_error(test,pred)
    performance["Model"].append(listt)
    performance["RMSE"].append(rmse)
    performance["MaPe"].append(mape)
    performance["Lag"].append(order)
    performance["Test"].append(test_obs)
    perf=pd.DataFrame(performance)
    return perf,result,pred

In [13]:
listt=["Close","High","Open","Low"]
#listt=["AQI_calculated","PM10","PM2.5","NOx","NO2","NO","NH3","SO2","CO",'year']


In [14]:
perf,result,pred=cominbation(data1,listt)

['Close', 'High', 'Open', 'Low']
Order = 1
AIC:  -41.576332503726654
BIC:  -41.52447100279196

Order = 2
AIC:  -41.8996567598384
BIC:  -41.80627099960226

Order = 3
AIC:  -41.89668108122695
BIC:  -41.76173985728738

Order = 4
AIC:  -41.895561823716704
BIC:  -41.7190338935332

Order = 5
AIC:  -41.9246166173129
BIC:  -41.70647070014241

Order = 6
AIC:  -42.00676954931895
BIC:  -41.74697432615166

Order = 7
AIC:  -42.063048592450706
BIC:  -41.761572705945674

Order = 8
AIC:  -42.058911836473904
BIC:  -41.71572389089464

Order = 9
AIC:  -42.07506774212317
BIC:  -41.690136303272986

Order = 10
AIC:  -42.096303169891954
BIC:  -41.66959676504925



In [15]:
data1

,Open,High,Low,Close
0,0.849900,0.845408,0.856100,0.854203
1,0.856394,0.967399,0.861040,0.974481
2,0.988480,0.996439,0.984959,0.986240
3,0.985483,0.968105,0.960760,0.956749
4,0.955669,0.975319,0.955033,0.967132
...,...,...,...,...
2220,0.095842,0.096329,0.096510,0.097323
2221,0.097777,0.095745,0.096951,0.096041
2222,0.096466,0.093934,0.095252,0.094821
2223,0.094031,0.105047,0.093143,0.105673


In [16]:
perf

,Model,RMSE,MaPe,Lag,Test
0,"[Close, High, Open, Low]",0,0.088105,11,28
